# Webscraping Bank Product Reviews
BoA Datasource = https://wallethub.com/profile/bank-of-america-13000450i

In [6]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import  Options
import pandas as pd
import time
import datetime
from random import uniform, choice

In [7]:
def get_proxies():
    url = "https://www.us-proxy.org//"
    # get the HTTP response and construct soup object
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    proxies = []
    for row in soup.find("table").findAll("tr"):
        tds = row.find_all("td")
        # check allows https and is elite class
        try:
            # print(tds[6], tds[4])
            if tds[6].text == 'yes' and tds[4].text == 'elite proxy':
                ip = tds[0].text.strip()
                port = tds[1].text.strip()
                host = f"{ip}:{port}"
                proxies.append(host)
        except IndexError:
            continue
    return proxies

def select_proxy(proxies):
    proxy = choice(proxies)
    return proxy

In [142]:
# Use Driver to fetch website
# wells-fargo-13007950i
# bank-of-america-13000450i
url = "https://wallethub.com/profile/wells-fargo-13007950i" 
DRIVER_PATH = "/Users/jesidacosta/OneDrive - University of South Florida/ISM6930/group_project/chromedriver"

In [116]:
def create_driver_session(bank="Wells Fargo",headless=True):
    # Get Free Proxy
    proxies = get_proxies()
    proxy = select_proxy(proxies) # IP:PORT or HOST:PORT
    # Set Proxy address to fetch new data
    options = Options()
    options.add_argument('--proxy-server=%s' % proxy)
    # No need to see page
    options.headless = headless
    # Create driver
    driver = webdriver.Chrome(DRIVER_PATH, options=options)
    # Testing it is functioning URL
    driver.get(url)
    if bank not in driver.title:
        driver.quit()
        create_driver_session(headless=False)
    else:    
        return driver

In [141]:
driver = create_driver_session(headless=False)
# driver.get(url)

In [108]:
# Create Headless Session (not GUI)
options = Options()
proxies = get_proxies()
proxy = select_proxy(proxies) # IP:PORT or HOST:PORT
options.add_argument('--proxy-server=%s' % proxy)
options.headless = False
# Create Driver
driver = webdriver.Chrome(DRIVER_PATH, options=options)
print('Proxy IP:', proxy)
driver.get(url + "?p=" + "1")

# Fetch HTML using Beautiful Soup
time.sleep(1)
# soup = BeautifulSoup(driver.page_source, 'lxml', from_encoding='UTF-8')

# for review in soup.find_all(class_='rvtab-citem'):
#     author_name = review.find(class_='rvtab-ci-name').text.strip()
#     date = review.find('time')['datetime']
#     comment = review.find(class_='rvtab-ci-content').text
#     print(date, author_name, comment, '\n')

Proxy IP: 207.244.170.103:8118


In [73]:
# driver.find_element_by_xpath('//section/nav/a[2]').click()
# soup =BeautifulSoup(driver.page_source, 'lxml', from_encoding='UTF-8')

NoneType

In [121]:
def get_html_file(url, driver, page=None):
    if page != None:
        url += "?p=" + page
    # inserting random time lapse to try to prevent bot detection
    random_value = uniform(1.5,4.0)
    print('Waited', random_value, 'seconds')
    time.sleep(random_value) 
    # fetch website
    driver.get(url)
    # get html
    soup = BeautifulSoup(driver.page_source, 'lxml', from_encoding='UTF-8')
    return soup

In [27]:
html = get_html_file(url, driver)
page_position = html.find(class_='rvtab-pag-pos').text
current_page = page_position.split()[0]
last_page = page_position.split()[2]

Waited 1.4975211732590288 seconds


/Users/jesidacosta/.local/share/virtualenvs/ISM6930-CY7GBNHA/lib/python3.9/site-packages/bs4/__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [28]:
html.title

<title>Bank of America Reviews: 39,327 User Ratings</title>

In [29]:
print(current_page, last_page)

1 283


In [30]:
# Create DataFrame for storage
col_names = ['date', 'name', 'user_id', 'verified', 'product', 'review_comment', 'stars', 'review']
data = pd.DataFrame(columns=col_names)

In [31]:
def fetch_page_reviews(html, df):
    for review in html.find_all(class_='rvtab-citem'):  
        # Initialize Values
        stars = 0
        verified = True
        review_comment = False
        # Name & UserName
        author_name = review.find(class_='rvtab-ci-name').text.strip()
        nickname = review.find(class_='rvtab-ci-nickname').text[1:]
        # Date of Review
        date = review.find('time')['datetime']
        # Product Reviewed
        if review.find(class_='rvtab-ci-category') is not None:
            product = review.find(class_='rvtab-ci-category').text.replace('Product:','').strip()
        else:
            # prints comments that don't belong to specific product, probably comments to previous review
            print(author_name, nickname, date, verified, stars)
            review_comment = True
            product = None
        # Review Comment
        comment = review.find(class_='rvtab-ci-content').text
        # Count number of stars
        for star in review.find_all('path'):
            if star['fill'] == '#4ae0e1':
                stars += 1
        if review.find(class_='rvtab-ci-verified') is None:
            verified = False
            # print(author_name, nickname, date, verified, stars)
            # print(product)
            # print('\t', comment)
        # temporary to add row by row
        row = [[date, author_name, nickname, verified, product, review_comment, stars, comment]]
        temp = pd.DataFrame(row, columns=df.columns)
        # add to main dataframe
        df = df.append(temp, ignore_index=True)
    return df

In [32]:
data = fetch_page_reviews(html, data)
data

,date,name,user_id,verified,product,review_comment,stars,review
0,2021-09-05,kupkayk,kupkayk,False,Bank of America Checking,False,1,If I could give negative stars I would. So I t...
1,2021-09-05,whuser15716867,whuser15716867,True,Bank of America Credit Cards,False,5,Bank of America's customer service make it ver...
2,2021-09-04,Hobed,hobed,True,Bank of America Credit Cards,False,3,"Overall, the experience has been mostly positi..."
3,2021-09-02,Jonathon,jonathon_lr_bell,True,Bank of America Credit Cards,False,3,Nice low rate and long 0 interest entry promot...
4,2021-09-01,Patricia,patriciad_145,True,Bank of America Credit Cards,False,5,Have been with BOA for about 15 years now. Geo...
5,2021-08-31,Richard,moredock,True,Bank of America Credit Cards,False,4,The card has had a relatively low interest rat...
6,2021-08-29,Mohammed,mohammeda_10,True,Bank of America Credit Cards,False,5,Bank of America credit card sucks I wouldn’t r...
7,2021-08-29,sonjapenner,sonjapenner,False,Bank of America Credit Cards,False,1,They started out great but I recently have bee...
8,2021-08-28,kelvinh_12,kelvinh_12,False,Bank of America Business Services,False,1,Bank of America is the worst bank to bank with...
9,2021-08-27,fayesimp5,fayesimp5,True,Bank of America Credit Cards,False,5,They were a good credit card for building cred...


In [33]:
# for review in html.find_all(class_='rvtab-citem'):  
#     # Initialize Values
#     stars = 0
#     verified = True
#     # Name & UserName
#     author_name = review.find(class_='rvtab-ci-name').text.strip()
#     nickname = review.find(class_='rvtab-ci-nickname').text[1:]
#     # Date of Review
#     date = review.find('time')['datetime']
#     # Product Reviewed
#     product = review.find(class_='rvtab-ci-category').text.replace('Product:','').strip()
#     # Review Comment
#     comment = review.find(class_='rvtab-ci-content').text
#     # Count number of stars
#     for star in review.find_all('path'):
#         if star['fill'] == '#4ae0e1':
#             stars += 1
#     if review.find(class_='rvtab-ci-verified') is None:
#         verified = False
#         # print(author_name, nickname, date, verified, stars)
#         # print(product)
#         # print('\t', comment)
#     # temporary to add row by row
#     row = [[date, author_name, nickname, verified, product, stars, comment]]
#     temp = pd.DataFrame(row, columns=col_names)
#     # add to main dataframe
#     data = data.append(temp, ignore_index=True)
# data

In [34]:
# TODO:
# 2 potential breaks: 1) if too many requests 2) no windown exists exception
# IF continues to crash try the clicking option instead of a new request

In [144]:
def scrape_reviews(url, driver, start_page=None, data=None):
    if data is None:
        # Create DataFrame for storage
        col_names = ['date', 'name', 'user_id', 'verified', 'product', 'review_comment', 'stars', 'review']
        data = pd.DataFrame(columns=col_names)
    # attempts counter
    attempts = 0
    # fetch starting page
    html = get_html_file(url, driver)
    # get page positions
    page_position = html.find(class_='rvtab-pag-pos').text
    if start_page is None:
        current_page = page_position.split()[0]
    else:
        current_page = str(start_page)
    last_page = page_position.split()[2]
    while int(current_page) <= int(last_page):
        print(current_page)   
        try:
            html = get_html_file(url, driver, current_page)
        except Exception as e:
            if attempts > 3:
                break
            print('Exception trigerred:', e)
            driver = create_driver_session(headless=False)
            attempts += 1
        # check comment review exists in page - else bot may be blocked
        if html.find_all(class_='rvtab-citem'):
            # add page reviews
            data = fetch_page_reviews(html, data)
            # get new page number
            current_page = html.find(class_='rvtab-pag-pos').text.split()[0]
            # increase page
            current_page = str(int(current_page) + 1)
        elif html.title.text == 'IP Block':
            print('No more comments retrieved - Bot Blocked')
            # print('Creating driver with new IP address')
            # driver = create_driver_session(headless=False)
            if attempts > 3:
                break
            attempts += 1
    driver.close()
    return data, current_page

In [145]:
data, page = scrape_reviews(url,driver)
data

Waited 3.383418157980321 seconds


/Users/jesidacosta/.local/share/virtualenvs/ISM6930-CY7GBNHA/lib/python3.9/site-packages/bs4/__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


1
Waited 2.194527017247564 seconds
aaron saxton aaron_e_saxton 2020-01-04 True 0
Kimberly Askew kimberlyaskew 2021-07-28 True 0
2
Waited 3.2216937353689006 seconds
3
Waited 1.9106382163908937 seconds
4
Waited 2.0235116907376733 seconds
Corey coreyr6 2021-07-22 True 0
5
Waited 1.7188082445159878 seconds
Kathy kathy_tinney 2021-07-19 True 0
6
Waited 3.439025365097044 seconds
Timo Marx globalwarmingis 2021-07-05 True 0
7
Waited 3.206999002238994 seconds
8
Waited 2.987653727904415 seconds
9
Waited 1.7802742880229356 seconds
ELIZABETH ANDERSON lzzy_marie82 2021-06-03 True 0
ELIZABETH ANDERSON lzzy_marie82 2021-06-03 True 0
10
Waited 2.4631076202143882 seconds


In [138]:
data, page = scrape_reviews(url,driver,page,data)
data

Waited 3.1625246660874176 seconds


/Users/jesidacosta/.local/share/virtualenvs/ISM6930-CY7GBNHA/lib/python3.9/site-packages/bs4/__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


166
Waited 3.6100653429437193 seconds
167
Waited 1.5616581492519683 seconds
AL Diaz ald3 2014-08-21 True 0
Asia Wang asia.wang.1 2014-07-26 True 0
Clay Ermel clay.ermel 2015-01-18 True 0
Snoopy Longfellow snoopyl 2015-09-05 True 0
Gina Hall gina_hall 2016-06-16 True 0
168
Waited 2.7118539996609217 seconds
Gina Hall gina_hall 2016-06-16 True 0
169
Waited 3.9795991392557863 seconds
Jamey Watkins Jwatkins 2014-05-21 True 0
170
Waited 1.768575577305099 seconds
171
Waited 2.540202173701145 seconds
172
Waited 3.011299547259366 seconds
173
Waited 2.9996801979887904 seconds
174
Waited 3.413272649916765 seconds
175
Waited 3.7430976780192085 seconds
176
Waited 3.268742425480209 seconds
177
Waited 1.7145308895533728 seconds
178
Waited 2.79832538989505 seconds
179
Waited 2.8042849821222813 seconds
180
Waited 2.8461367504796597 seconds
181
Waited 2.947835480546073 seconds
182
Waited 2.8744268343553876 seconds
183
Waited 3.7460722454864652 seconds
184
Waited 2.9190276956169905 seconds
Angela Siegfri

,date,name,user_id,verified,product,review_comment,stars,review
0,2021-09-05,kupkayk,kupkayk,False,Bank of America Checking,False,1,If I could give negative stars I would. So I t...
1,2021-09-05,whuser15716867,whuser15716867,True,Bank of America Credit Cards,False,5,Bank of America's customer service make it ver...
2,2021-09-04,Hobed,hobed,True,Bank of America Credit Cards,False,3,"Overall, the experience has been mostly positi..."
3,2021-09-02,Jonathon,jonathon_lr_bell,True,Bank of America Credit Cards,False,3,Nice low rate and long 0 interest entry promot...
4,2021-09-01,Patricia,patriciad_145,True,Bank of America Credit Cards,False,5,Have been with BOA for about 15 years now. Geo...
...,...,...,...,...,...,...,...,...
2930,2011-09-28,Joe Weider,joe_weider,False,Bank of America Credit Cards,False,5,"So, in the latest installment of my little cre..."
2931,2011-09-28,Chris Kelly,chriskelly06,False,Bank of America Credit Cards,False,1,"Here's a gem:""No pre-set spending limit does n..."
2932,2011-09-27,Joe Weider,joe_weider,False,Bank of America Credit Cards,False,5,"Ok, so I’ve been interested in credit cards fo..."
2933,2011-09-14,Rolling Blunts,daddy.chris17,False,Bank of America Credit Cards,False,5,I have the platinum plus I must say its one of...


In [129]:
page

'166'

In [139]:
# Save Document as Pandas
DATE = datetime.datetime.now().date()
data.to_pickle("wellsfargo_wallethub_reviews_{}.pkl".format(DATE))
data.to_json('wellsfargo_wallethub_reviews.json', orient='records')
data.to_csv("wellsfargo_wallethub_reviews_{}.csv".format(DATE), index=False, encoding='utf-8')

In [37]:
# Encoding seems to be a problem in excel not in 
data.iloc[26,:].comment

'I make payments on time I missed one payment and BoA did not waive my late fees. WORST CUSTOMER SERVICE EVER! Very rude reps don’t even want to listen to you. I have been a loyal customer for years and I forgot to make a payment by 2 days and it was my first time getting late fee they did not even consider to waive it off!!!'

In [43]:
df = pd.read_pickle('boa_wallethub_reviews_2021-09-05.pkl')

In [45]:
df.iloc[22].comment

'The card has a $39 per year fee. I’ve had it almost 7 years still a low limit - $2000. My other cards have MUCH higher limits. I have great credit.  I keep it to use for internet purchases, it’s easily monitored and keeps my other higher limit cards safe from the web.'

In [44]:
data

,date,name,user_id,verified,product,review_comment,stars,review
0,2021-09-05,kupkayk,kupkayk,False,Bank of America Checking,False,1,If I could give negative stars I would. So I t...
1,2021-09-05,whuser15716867,whuser15716867,True,Bank of America Credit Cards,False,5,Bank of America's customer service make it ver...
2,2021-09-04,Hobed,hobed,True,Bank of America Credit Cards,False,3,"Overall, the experience has been mostly positi..."
3,2021-09-02,Jonathon,jonathon_lr_bell,True,Bank of America Credit Cards,False,3,Nice low rate and long 0 interest entry promot...
4,2021-09-01,Patricia,patriciad_145,True,Bank of America Credit Cards,False,5,Have been with BOA for about 15 years now. Geo...
5,2021-08-31,Richard,moredock,True,Bank of America Credit Cards,False,4,The card has had a relatively low interest rat...
6,2021-08-29,Mohammed,mohammeda_10,True,Bank of America Credit Cards,False,5,Bank of America credit card sucks I wouldn’t r...
7,2021-08-29,sonjapenner,sonjapenner,False,Bank of America Credit Cards,False,1,They started out great but I recently have bee...
8,2021-08-28,kelvinh_12,kelvinh_12,False,Bank of America Business Services,False,1,Bank of America is the worst bank to bank with...
9,2021-08-27,fayesimp5,fayesimp5,True,Bank of America Credit Cards,False,5,They were a good credit card for building cred...
